# HE Team LLM Assistant - API Examples

**Version**: 2.2.0

**Base URL**: `http://localhost:8000` (change to your server IP if needed)

This notebook demonstrates all available API endpoints.

## Prerequisites

1. Backend server running: `python run_backend.py`
2. Valid user credentials

## Table of Contents

1. [Client Setup](#client-setup)
2. [Authentication](#authentication)
3. [Normal Chat](#normal-chat)
4. [RAG Chat](#rag-chat)
5. [Web Search Chat](#web-search)
6. [JSON Analysis](#json-analysis)
7. [File Upload & Analysis](#file-analysis)
8. [Conversation Management](#conversations)
9. [System Health](#health)

## 1. Client Setup

Complete Python client with all API methods.

In [ ]:
import requests
import json
from pathlib import Path

class LLMClient:
    """Complete API client for HE Team LLM Assistant."""
    
    def __init__(self, base_url: str = "http://localhost:8000"):
        self.base_url = base_url
        self.session_token = None
        self.headers = {"Content-Type": "application/json"}

    def login(self, username: str, password: str) -> bool:
        """Login and obtain session token."""
        response = requests.post(
            f"{self.base_url}/api/auth/login",
            headers=self.headers,
            json={"username": username, "password": password}
        )
        if response.status_code == 200:
            data = response.json()
            self.session_token = data["session_token"]
            self.headers["Authorization"] = f"Bearer {self.session_token}"
            print(f"✓ Login successful")
            print(f"  User: {data['user']['username']}")
            print(f"  Role: {data['user']['role']}")
            return True
        else:
            print(f"✗ Login failed: {response.status_code}")
            return False
    
    def logout(self) -> bool:
        """Logout and invalidate token."""
        response = requests.post(
            f"{self.base_url}/api/auth/logout",
            headers=self.headers
        )
        if response.status_code == 200:
            self.session_token = None
            del self.headers["Authorization"]
            print("✓ Logged out")
            return True
        return False
    
    def chat(self, message: str, session_id=None, chat_type="normal", 
             json_data=None, json_path=None, temperature=None) -> dict:
        """Send a message to the LLM API.
        
        Args:
            message: Your question/message
            session_id: Continue existing conversation (optional)
            chat_type: "normal", "rag", "web_search", or "json"
            json_data: JSON object for analysis (for json mode)
            json_path: Path to JSON file (for json mode)
            temperature: Override default temperature (0.0-1.0)
        """
        endpoint_map = {
            "normal": "/api/chat",
            "rag": "/api/chat/rag",
            "web_search": "/api/chat/web-search",
            "json": "/api/chat/with-json"
        }
        endpoint = endpoint_map.get(chat_type, "/api/chat")

        if chat_type == "json":
            # Load JSON from file if path provided
            if json_path and not json_data:
                with open(json_path, "r", encoding="utf-8") as f:
                    json_data = json.load(f)
            
            payload = {
                "message": message,
                "json_data": json_data,
                "session_id": session_id,
                "temperature": temperature if temperature is not None else 0.0
            }
        else:
            payload = {"message": message}
            if session_id:
                payload["session_id"] = session_id
            if temperature is not None:
                payload["temperature"] = temperature

        response = requests.post(
            f"{self.base_url}{endpoint}",
            headers=self.headers,
            json=payload,
            timeout=600
        )
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"✗ Error: {response.status_code}")
            print(response.text)
            return None
    
    def upload_file(self, file_path: str) -> dict:
        """Upload a file for analysis."""
        with open(file_path, 'rb') as f:
            files = {'file': f}
            headers = {"Authorization": self.headers.get("Authorization")}
            response = requests.post(
                f"{self.base_url}/api/files/upload",
                headers=headers,
                files=files
            )
        
        if response.status_code == 200:
            data = response.json()
            print(f"✓ File uploaded: {data['original_name']}")
            print(f"  File ID: {data['file_id']}")
            return data
        else:
            print(f"✗ Upload failed: {response.status_code}")
            return None
    
    def list_files(self) -> list:
        """List all uploaded files."""
        response = requests.get(
            f"{self.base_url}/api/files",
            headers=self.headers
        )
        return response.json().get("files", [])
    
    def analyze_file(self, file_id: str, question: str, session_id=None) -> dict:
        """Ask questions about an uploaded file."""
        response = requests.post(
            f"{self.base_url}/api/files/{file_id}/read",
            headers=self.headers,
            json={"question": question, "session_id": session_id}
        )
        return response.json() if response.status_code == 200 else None
    
    def delete_file(self, file_id: str) -> bool:
        """Delete an uploaded file."""
        response = requests.delete(
            f"{self.base_url}/api/files/{file_id}",
            headers=self.headers
        )
        return response.status_code == 200
    
    def list_sessions(self) -> list:
        """List all conversation sessions."""
        response = requests.get(
            f"{self.base_url}/api/chat/sessions",
            headers=self.headers
        )
        return response.json().get("sessions", [])
    
    def get_session(self, session_id: str) -> dict:
        """Get conversation history."""
        response = requests.get(
            f"{self.base_url}/api/chat/sessions/{session_id}",
            headers=self.headers
        )
        return response.json()
    
    def delete_session(self, session_id: str) -> bool:
        """Delete a conversation session."""
        response = requests.delete(
            f"{self.base_url}/api/chat/sessions/{session_id}",
            headers=self.headers
        )
        return response.json().get("deleted", False)
    
    def health_check(self) -> dict:
        """Check system health."""
        response = requests.get(f"{self.base_url}/health")
        return response.json()
    
    def get_config(self) -> dict:
        """Get current configuration."""
        response = requests.get(f"{self.base_url}/api/config")
        return response.json()
    
    def list_models(self) -> list:
        """List available LLM models."""
        response = requests.get(f"{self.base_url}/api/models")
        return response.json().get("models", [])

print("✓ LLMClient class loaded")

## 2. Authentication

Initialize client and login.

In [ ]:
# Create client instance
# Change base_url to your server IP if needed
client = LLMClient(base_url="http://localhost:8000")

# Login with your credentials
client.login("guest", "guest_test1")

## 3. System Health Check

Check if the system is running properly.

In [ ]:
health = client.health_check()
print(json.dumps(health, indent=2))

# Check specific features
print(f"\nStatus: {health['status']}")
print(f"Model: {health.get('model', 'Unknown')}")
print(f"Web Search: {'Enabled' if health.get('web_search_enabled') else 'Disabled'}")
print(f"RAG System: {'Ready' if health.get('rag_enabled') else 'Not configured'}")

## 4. Normal Chat

Basic conversation with the LLM.

In [ ]:
# Start a new conversation
result = client.chat("Hello! Can you explain what you can help me with?")

if result:
    session_id = result["session_id"]
    print("AI Response:")
    print(result['response'])
    print(f"\nSession ID: {session_id}")

In [ ]:
# Continue the conversation
result = client.chat(
    "What programming languages can you help with?",
    session_id=session_id
)

print("AI Response:")
print(result['response'])

## 5. RAG Chat

Search knowledge base and answer based on documents.

In [ ]:
# RAG-enabled chat
result = client.chat(
    "What documents do we have about warpage?",
    chat_type="rag"
)

if result:
    print("AI Response (with RAG):")
    print(result['response'])

## 6. Web Search Chat

Get current information from the internet.

In [ ]:
# Web search chat
result = client.chat(
    "What are the latest developments in large language models?",
    chat_type="web_search"
)

if result:
    print("AI Response (with web search):")
    print(result['response'])
    print(f"\nKeyword extraction used: {result.get('keyword_extraction_used', False)}")
    print(f"Successful query: {result.get('successful_query', 'N/A')}")
    
    if result.get('search_results'):
        print(f"\nSearch results found: {len(result['search_results'])}")
        for i, sr in enumerate(result['search_results'][:3], 1):
            print(f"\n{i}. {sr['title']}")
            print(f"   URL: {sr['url']}")

## 7. JSON Analysis

High-accuracy JSON analysis with zero-hallucination mode.

In [ ]:
# Example 1: Inline JSON data
json_data = {
    "materials": [
        {"id": "ABC123", "name": "Material A", "warpage": 0.45, "temperature": 245},
        {"id": "XYZ789", "name": "Material B", "warpage": 1.23, "temperature": 280},
        {"id": "DEF456", "name": "Material C", "warpage": 0.89, "temperature": 260}
    ]
}

result = client.chat(
    "Which material has the lowest warpage and what is its temperature?",
    chat_type="json",
    json_data=json_data
)

if result:
    print("AI Response:")
    print(result['response'])
    
    print("\n" + "="*60)
    print("Numeric Summary (auto-generated):")
    print("="*60)
    print(result.get('numeric_summary', 'N/A'))
    
    if result.get('validation_notes'):
        vn = result['validation_notes']
        print("\n" + "="*60)
        print("Validation:")
        print("="*60)
        print(f"Validated: {vn.get('validated', False)}")
        if vn.get('warnings'):
            print("Warnings:", vn['warnings'])
        if vn.get('info'):
            print("Info:", vn['info'])

In [ ]:
# Example 2: JSON from file
# Make sure the file exists at this path
json_file_path = "./data/sample_data.json"  # Change this to your JSON file path

# Uncomment if you have a JSON file:
# result = client.chat(
#     "Summarize the data and find the maximum value",
#     chat_type="json",
#     json_path=json_file_path
# )
# print(result['response'])

## 8. File Upload & Analysis

Upload files and ask questions about them.

In [ ]:
# List currently uploaded files
files = client.list_files()
print(f"Uploaded files: {len(files)}")
for f in files:
    print(f"  - {f['original_name']} (ID: {f['file_id']})")

In [ ]:
# Upload a file
# Change this to your file path
file_path = "./sample_document.pdf"  # PDF, DOCX, TXT, etc.

# Uncomment if you have a file:
# upload_result = client.upload_file(file_path)
# if upload_result:
#     file_id = upload_result['file_id']
#     print(f"File uploaded with ID: {file_id}")

In [ ]:
# Analyze uploaded file
# Use the file_id from upload or list_files()

# Uncomment and set your file_id:
# file_id = "your-file-id-here"
# result = client.analyze_file(
#     file_id,
#     "What is the main topic of this document?"
# )
# if result:
#     print("AI Response:")
#     print(result['response'])

## 9. Conversation Management

List, view, and manage conversations.

In [ ]:
# List all sessions
sessions = client.list_sessions()
print(f"Total sessions: {len(sessions)}")
for s in sessions[:5]:  # Show first 5
    print(f"  Session: {s['session_id']}")
    print(f"    Messages: {s.get('message_count', 'Unknown')}")
    print(f"    Created: {s.get('created_at', 'Unknown')}")

In [ ]:
# View specific session history
if session_id:  # From earlier chat
    history = client.get_session(session_id)
    print(f"Session: {session_id}")
    print(f"Messages: {len(history.get('history', []))}\n")
    
    for msg in history.get('history', []):
        role = msg['role'].upper()
        content = msg['content'][:100] + "..." if len(msg['content']) > 100 else msg['content']
        print(f"{role}: {content}\n")

## 10. Available Models

List LLM models available on the server.

In [ ]:
# List available models
models = client.list_models()
print(f"Available models: {len(models)}\n")
for m in models:
    print(f"  - {m['name']}")
    print(f"    Size: {m.get('size', 'Unknown')}")
    print()

## 11. Cleanup

Logout when done.

In [ ]:
# Logout
client.logout()

## Summary

This notebook demonstrated:

1. ✅ Authentication (login/logout)
2. ✅ Health checks
3. ✅ Normal chat conversations
4. ✅ RAG-enabled chat (knowledge base)
5. ✅ Web search integration
6. ✅ High-accuracy JSON analysis
7. ✅ File upload and analysis
8. ✅ Conversation management
9. ✅ Model listing

### Next Steps

- See [API_REFERENCE.md](API_REFERENCE.md) for complete API documentation
- Check [docs/QUICK_START.md](docs/QUICK_START.md) for getting started guide
- Review [docs/README_SEPARATED_SERVERS.md](docs/README_SEPARATED_SERVERS.md) for deployment

### Support

For questions or issues, contact: s.hun.lee